In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
import psycopg
import sqlite3
import plotly.express as px
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sqlalchemy import create_engine    
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
postgrespassword = os.getenv('POSTGRES_PASSWORD')

In [2]:
ct = contrans()

In [3]:
server, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD)

In [5]:
server.execute("CREATE DATABASE contrans;")

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost database=postgres) at 0x171dec1d0>

In [ ]:
#read csv
# world = pd.read_csv('world.csv')
# feed the csv into the database
# world.to_sql('world', engine, if_exists='replace', index=False)

In [6]:
myquery = '''
SELECT *
FROM members
'''
pd.read_sql_query(myquery, con=engine)  

ProgrammingError: (psycopg.errors.UndefinedTable) relation "members" does not exist
LINE 3: FROM members
             ^
[SQL: 
SELECT *
FROM members
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [6]:
r =ct.summarize_news('N000188', engine)

In [8]:
r.json()

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [31]:
news_query = 'Tim Kaine congress'
r = requests.get('https://newsapi.org/v2/everything',
            params={'q': news_query, 'apiKey': ct.newskey, 'searchIn': 'content'},
            headers={'User-Agent': ct.useragent})
r.json()

{'status': 'ok',
 'totalResults': 32,
 'articles': [{'source': {'id': 'breitbart-news', 'name': 'Breitbart News'},
   'author': 'Bradley Jaye, Bradley Jaye',
   'title': 'Tim Kaine Projected to Win Reelection to Senate',
   'description': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\nThe post Tim Kaine Projected to Win Reelection to Senate appeared first on Breitbart.',
   'url': 'https://www.breitbart.com/politics/2024/11/06/tim-kaine-projected-to-win-reelection-to-senate/',
   'urlToImage': 'https://media.breitbart.com/media/2024/11/tim-paine-640x335.jpg',
   'publishedAt': '2024-11-06T08:06:54Z',
   'content': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\r\nKaines seat was considered relatively safe, although Republicans were… [+2104 chars]'},
  {'source': {'id': 'new-york-magazine', 'name': 'New York Magazine'},
  

In [36]:
content = [x['title']   for x in r.json()['articles']]

In [37]:
input_text = ' '.join(content)

In [38]:
ct.get_summary_text(input_text)

"Tim Kaine Projected to Win Reelection to Senate Trump Can Prosecute Anybody He Wants, Transition Leader Says Why That Kamala Harris Saturday Night Live Appearance Triggered an FCC Response Links 10/23/2024 Columbia-Class and Virginia-Class Submarines: Overbudget and Behind Schedule (Why?)LIVE: Northern Virginia and statewide election results In Virginia, a Vietnamese American Community Divided Falls Church elected leaders wary of Trump return, GOP dominance in Congress Morning Notes for October 25, 2024 Franks: Dems fumbled by passing over Hillary for Harris Trump’s mainstream picks for top foreign policy posts could reassure allies — and worry China An hour-by-hour guide to watching the races on election night Trump's mainstream picks for top foreign policy posts could reassure allies — and worry China Trump's mainstream picks for top foreign policy posts could reassure allies - and worry China Sen. Tim Kaine wins 3rd term in Washington Rep. Gerry Connolly reveals cancer treatment af